# Clustering Fractions – Profils d’erreurs (RQ2)

Objectif : **constituer des groupes de besoin** à partir des **profils d’erreurs** des élèves sur les **fractions**.

Ce notebook suppose un fichier : `data/students/responses.csv` avec au minimum :

- `student_id`
- `error_tags` (tags séparés par `|` si plusieurs)

Exemple :
```csv
student_id,item_id,skill,is_correct,error_tags
E01,Q1,add_fractions,0,add_denominators|no_common_denominator
```

Sorties produites :
- choix d’un **K** (silhouette)
- clusters + **interprétation pédagogique** (tags dominants)
- export `data/students/groups_of_need.csv`


In [ ]:
# ✅ 1) Imports & chemins
from pathlib import Path
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

BASE_DIR = Path.cwd()
RESPONSES_CSV = Path(os.getenv("RESPONSES_CSV", str(BASE_DIR / "data" / "students" / "responses.csv")))
OUTPUT_CSV = Path(os.getenv("GROUPS_OUTPUT_CSV", str(BASE_DIR / "data" / "students" / "groups_of_need.csv")))

print("RESPONSES_CSV:", RESPONSES_CSV)
print("OUTPUT_CSV   :", OUTPUT_CSV)


In [ ]:
# ✅ 2) Charger les données
if not RESPONSES_CSV.exists():
    raise FileNotFoundError(f"Fichier introuvable: {RESPONSES_CSV}")

df = pd.read_csv(RESPONSES_CSV)
print("Shape:", df.shape)
df.head()


## 3) Nettoyage / parsing des `error_tags`

- `error_tags` peut être vide si l’item est correct.
- on transforme en liste de tags par ligne
- on agrège par élève pour obtenir un profil (fréquences de tags)


In [ ]:
# ✅ 3) Parser les tags
def split_tags(x):
    if pd.isna(x):
        return []
    x = str(x).strip()
    if x == "":
        return []
    return [t.strip() for t in x.split("|") if t.strip()]

df["tags_list"] = df.get("error_tags", pd.Series([""]*len(df))).apply(split_tags)

# Quelques stats rapides
n_empty = (df["tags_list"].apply(len) == 0).sum()
print("Lignes sans tags:", n_empty, "/", len(df))

df[["student_id"] + [c for c in df.columns if c in ("item_id","skill","is_correct","error_tags","tags_list")]].head(10)


## 4) Construction de la matrice `élève × tag`

On construit une matrice de comptage (ou fréquence) :  
- lignes = élèves  
- colonnes = tags d’erreurs  
- valeurs = nombre d’occurrences (par défaut)


In [ ]:
# ✅ 4) Matrice élève x tag (comptage)
students = sorted(df["student_id"].astype(str).unique().tolist())

# Liste des tags présents
all_tags = sorted({t for tags in df["tags_list"] for t in tags})
print("Nb élèves:", len(students))
print("Nb tags  :", len(all_tags))
print("Exemples tags:", all_tags[:15])

# Construire la matrice de comptage
student_index = {s:i for i,s in enumerate(students)}
tag_index = {t:j for j,t in enumerate(all_tags)}

X = np.zeros((len(students), len(all_tags)), dtype=float)

for _, row in df.iterrows():
    s = str(row["student_id"])
    i = student_index[s]
    for t in row["tags_list"]:
        j = tag_index[t]
        X[i, j] += 1.0

# Option: passer en fréquences par élève (utile si nb items très différent selon élèves)
row_sums = X.sum(axis=1, keepdims=True)
X_freq = np.divide(X, row_sums, out=np.zeros_like(X), where=row_sums!=0)

print("X shape:", X.shape)


## 5) Choisir le nombre de clusters (K)

On teste plusieurs K (ex: 2 → 8) et on calcule le **silhouette score**.  
⚠️ Il faut au moins 2 élèves et une variance minimale.

> Si tu as très peu d’élèves, ne cherche pas à sur-optimiser : l’interprétation pédagogique prime.


In [ ]:
# ✅ 5) Recherche du meilleur K (silhouette)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def try_k_values(X_used, k_min=2, k_max=8, random_state=42):
    scores = []
    for k in range(k_min, k_max+1):
        if X_used.shape[0] <= k:
            continue
        km = KMeans(n_clusters=k, n_init=10, random_state=random_state)
        labels = km.fit_predict(X_used)
        # silhouette nécessite au moins 2 clusters non vides
        if len(set(labels)) < 2:
            continue
        score = silhouette_score(X_used, labels)
        scores.append((k, score))
    return scores

scores = try_k_values(X_freq if X_freq.shape[1] > 0 else X)

scores


In [ ]:
# ✅ 5b) Visualiser les scores (simple)
import matplotlib.pyplot as plt

if scores:
    ks = [k for k,_ in scores]
    ss = [s for _,s in scores]
    plt.figure()
    plt.plot(ks, ss, marker="o")
    plt.xlabel("K (nombre de clusters)")
    plt.ylabel("Silhouette score")
    plt.title("Choix de K par silhouette")
    plt.show()
else:
    print("Pas assez de données pour calculer la silhouette (ajoute des élèves/items).")


## 6) Entraîner le clustering final

Par défaut :
- on prend le K avec silhouette max (si disponible)
- sinon on fixe K=3 (valeur raisonnable pour des groupes de besoin)


In [ ]:
# ✅ 6) Fit final
DEFAULT_K = 3

if scores:
    best_k = max(scores, key=lambda x: x[1])[0]
else:
    best_k = DEFAULT_K

print("K retenu:", best_k)

km = KMeans(n_clusters=best_k, n_init=20, random_state=42)
labels = km.fit_predict(X_freq if X_freq.shape[1] > 0 else X)

cluster_df = pd.DataFrame({
    "student_id": students,
    "cluster": labels
}).sort_values(["cluster", "student_id"]).reset_index(drop=True)

cluster_df.head(20)


## 7) Interprétation pédagogique des clusters

On calcule, pour chaque cluster, les **tags dominants** (fréquence moyenne la plus élevée).  
C’est cette partie qui te sert directement dans le mémoire : *cluster = profil d’erreurs = besoin de remédiation*.


In [ ]:
# ✅ 7) Tags dominants par cluster
tag_names = np.array(all_tags)

X_used = X_freq if X_freq.shape[1] > 0 else X
cluster_profiles = []

for c in sorted(cluster_df["cluster"].unique()):
    idx = cluster_df.index[cluster_df["cluster"] == c].to_list()
    # Moyenne des vecteurs d'erreurs dans le cluster
    mean_vec = X_used[idx].mean(axis=0) if len(idx) else np.zeros((X_used.shape[1],))
    top_idx = np.argsort(mean_vec)[::-1][:8]
    top_tags = [(tag_names[j], float(mean_vec[j])) for j in top_idx if mean_vec[j] > 0]
    cluster_profiles.append({
        "cluster": c,
        "n_students": int((cluster_df["cluster"] == c).sum()),
        "top_tags": top_tags
    })

cluster_profiles


In [ ]:
# ✅ 7b) Présentation lisible
for prof in cluster_profiles:
    print(f"\n=== Cluster {prof['cluster']} | n={prof['n_students']} ===")
    if not prof["top_tags"]:
        print("Aucun tag dominant (données insuffisantes ou erreurs non taguées).")
        continue
    for t, v in prof["top_tags"]:
        print(f"- {t}: {v:.3f}")


## 8) Export : groupes de besoin

On exporte `groups_of_need.csv` avec :
- `student_id`
- `cluster`
- `profile_summary` (tags dominants)

Ce fichier sert ensuite :
- à l’enseignant (groupes)
- à l’assistant (remédiation RAG par cluster)


In [ ]:
# ✅ 8) Construire un résumé de profil par cluster
summary_by_cluster = {}
for prof in cluster_profiles:
    tags_only = [t for t,_ in prof["top_tags"]][:5]
    summary_by_cluster[prof["cluster"]] = ", ".join(tags_only) if tags_only else ""

cluster_df["profile_summary"] = cluster_df["cluster"].map(summary_by_cluster)

# Sauvegarde
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
cluster_df.to_csv(OUTPUT_CSV, index=False)
print("✅ Exporté:", OUTPUT_CSV)
cluster_df.head(20)


## 9) Vérification qualitative (mini-liste par groupe)

Affiche les élèves par cluster.  
C’est la partie “enseignant-friendly”.


In [ ]:
# ✅ 9) Liste des élèves par groupe
for c in sorted(cluster_df["cluster"].unique()):
    group = cluster_df[cluster_df["cluster"] == c]["student_id"].tolist()
    print(f"\nGroupe {c} ({len(group)} élèves) – Profil: {summary_by_cluster.get(c,'')}")
    print(", ".join(group))


## 10) (Option) Lier cluster → remédiation via RAG

Ici, on montre comment une **question RAG** peut être générée à partir du profil du cluster,
afin de récupérer des ressources adaptées (cours, erreurs fréquentes, exercices gradués).

> Nécessite `rag_langchain.py` configuré et un corpus Fractions riche.


In [ ]:
# ✅ 10) Exemple: requête RAG par cluster (option)
try:
    from rag_langchain import rag_chain
    for prof in cluster_profiles:
        c = prof["cluster"]
        tags = [t for t,_ in prof["top_tags"]][:3]
        if not tags:
            continue
        q = (
            "Propose une remédiation courte sur les fractions pour les erreurs suivantes: "
            + ", ".join(tags)
            + ". Donne une explication + 2 exercices gradués."
        )
        r = rag_chain({"question": q})
        print(f"\n--- Remédiation pour cluster {c} ({', '.join(tags)}) ---")
        print((r.get('answer','') or '')[:1200])
except Exception as e:
    print("RAG non disponible dans ce notebook (ou corpus manquant):", e)
